In [ ]:
import os
import glob
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from scipy.signal import resample
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.model_selection import GroupKFold 



# =========================================================
# 0. 設定エリア
# =========================================================
BASE_DIR = r"C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet"

STAGE2_DIR = os.path.join(BASE_DIR, "stage2")
CSV_DIR    = os.path.join(BASE_DIR, "train_csvs")
TRAIN_META = os.path.join(BASE_DIR, "train.csv")
SAVE_DIR   = os.path.join(BASE_DIR, "seeed30")

# ハイパーパラメータ
BATCH_SIZE = 32
EPOCHS = 500
LR = 1e-3
PATIENCE = 20    # 停滞許容回数
SEED = 30
N_FOLDS = 5      # 5分割

# ★ブレーキを少し緩める（学習不足解消のため）
WEIGHT_DECAY = 1e-4 

# =========================================================
# 1. Utils
# =========================================================
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# =========================================================
# 2. Dataset Class
# =========================================================
class ECGDatasetRam(Dataset):
    def __init__(self, df, npy_dir, csv_dir, target_len=5000):
        self.target_len = target_len
        self.samples = [] 
        self.sample_ids = []  # ★追加: IDを記録するリスト
        if not os.path.exists(npy_dir):
            raise FileNotFoundError(f"Directory not found: {npy_dir}")
            
        target_ids = set(df['id'].astype(str).tolist())
        file_list = []
        all_files = glob.glob(os.path.join(npy_dir, "*.npy"))
        
        print(f"Scanning files in {npy_dir}...")
        for fpath in all_files:
            fname = os.path.basename(fpath)
            file_id = fname.split('-')[0]
            if file_id in target_ids:
                file_list.append((fpath, file_id))
        
        print(f"Found {len(file_list)} valid files. Loading ALL into RAM...")

        for fpath, sample_id in tqdm(file_list, desc="Loading Data"):
            processed = self.process_one_file(fpath, sample_id, csv_dir)
            if processed is not None:
                self.samples.append(processed)
                self.sample_ids.append(sample_id) # ★追加: 成功したデータのIDだけ記録
                
        print(f"Successfully loaded {len(self.samples)} samples.")
    
    def process_one_file(self, npy_path, sample_id, csv_dir):
        try:
            # ---------------------------------------------------------
            # 1. Input Load (新しい処理: 信号12ch + 確信度12ch = 24ch)
            # ---------------------------------------------------------
            data = np.load(npy_path)
            data = np.nan_to_num(data, nan=0.0)
            original_len = data.shape[1]
            if data.shape[0] != 13: return None

            # 信号用と確信度用の器を用意
            reconstructed_sig = np.zeros((12, original_len), dtype=np.float32)
            reconstructed_conf = np.zeros((12, original_len), dtype=np.float32)

            for i in range(4):
                sig_row = data[i]       # 信号
                conf_row = data[4+i]    # 確信度
                id_row = data[9+i]      # ID
                
                unique_ids = np.unique(id_row)
                for uid in unique_ids:
                    if 0 <= uid <= 11:
                        mask_ch = (id_row == uid)
                        reconstructed_sig[int(uid), mask_ch] = sig_row[mask_ch]
                        reconstructed_conf[int(uid), mask_ch] = conf_row[mask_ch]

            # ---------------------------------------------------------
            # 2. Target Load (以前の処理を復元: CSV読み込み)
            # ---------------------------------------------------------
            csv_path = os.path.join(csv_dir, f"{sample_id}.csv")
            if not os.path.exists(csv_path): return None

            target_df = pd.read_csv(csv_path)
            target_vals = target_df.values.T 
            mask_data = (~np.isnan(target_vals)).astype(np.float32)
            target_data = np.nan_to_num(target_vals, nan=0.0)

            # ---------------------------------------------------------
            # 3. Resample & Preprocessing (両方の処理を適用)
            # ---------------------------------------------------------
            
            # (A) Input Resample
            if reconstructed_sig.shape[1] != self.target_len:
                input_sig_final = resample(reconstructed_sig, self.target_len, axis=1)
                input_conf_final = resample(reconstructed_conf, self.target_len, axis=1)
            else:
                input_sig_final = reconstructed_sig
                input_conf_final = reconstructed_conf

            # (B) Target Resample
            if target_data.shape[1] != self.target_len:
                target_final = resample(target_data, self.target_len, axis=1)
                mask_final = resample(mask_data, self.target_len, axis=1)
            else:
                target_final = target_data
                mask_final = mask_data
            
            # (D) Concatenate (12ch + 12ch = 24ch)
            input_final = np.concatenate([input_sig_final, input_conf_final], axis=0)

            return (np.nan_to_num(input_final).astype(np.float32), 
                    np.nan_to_num(target_final).astype(np.float32), 
                    (mask_final > 0.5).astype(np.float32), 
                    original_len)
        except:
            return None

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        in_arr, tgt_arr, msk_arr, length = self.samples[idx]
        return (torch.from_numpy(in_arr), torch.from_numpy(tgt_arr), 
                torch.from_numpy(msk_arr), torch.tensor(length, dtype=torch.long))

# =========================================================
# 3. Model (修正版)
# =========================================================
class ResNet1d_UNet_Large(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Encoder
        self.enc1 = nn.Sequential(nn.Conv1d(24, 128, 7, 2, 3), nn.BatchNorm1d(128), nn.ReLU()) 
        self.enc2 = nn.Sequential(nn.Conv1d(128, 256, 3, 2, 1), nn.BatchNorm1d(256), nn.ReLU()) 
        self.enc3 = nn.Sequential(nn.Conv1d(256, 512, 3, 2, 1), nn.BatchNorm1d(512), nn.ReLU()) 
        self.enc4 = nn.Sequential(nn.Conv1d(512, 1024, 3, 2, 1), nn.BatchNorm1d(1024), nn.ReLU())
        
        # Decoder
        self.dec4 = nn.Sequential(nn.Conv1d(1024 + 512, 512, 3, 1, 1), nn.BatchNorm1d(512), nn.ReLU()) 
        self.dec3 = nn.Sequential(nn.Conv1d(512 + 256, 256, 3, 1, 1), nn.BatchNorm1d(256), nn.ReLU()) 
        self.dec2 = nn.Sequential(nn.Conv1d(256 + 128, 128, 3, 1, 1), nn.BatchNorm1d(128), nn.ReLU())
        self.dec1 = nn.Sequential(nn.Conv1d(128, 64, 3, 1, 1), nn.BatchNorm1d(64), nn.ReLU())
        
        self.final = nn.Conv1d(64, 12, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        
        d4 = torch.cat([torch.nn.functional.interpolate(e4, size=e3.shape[2]), e3], dim=1)
        d4 = self.dec4(d4)
        
        d3 = torch.cat([torch.nn.functional.interpolate(d4, size=e2.shape[2]), e2], dim=1)
        d3 = self.dec3(d3)
        
        d2 = torch.cat([torch.nn.functional.interpolate(d3, size=e1.shape[2]), e1], dim=1)
        d2 = self.dec2(d2)
        
        d1 = torch.nn.functional.interpolate(d2, size=x.shape[2])
        d1 = self.dec1(d1)
        
        out = self.final(d1)
        
        # ★修正箇所: 入力x(24ch)のうち、信号部分(先頭12ch)だけを取り出して足す
        input_signal = x[:, :12, :] 
        
        return input_signal + out

# =========================================================
# 4. Main Training Loop (5-Fold CV)
# =========================================================
def run_kfold_training():
    seed_everything(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using Device: {device}")
    
    if not os.path.exists(SAVE_DIR): os.makedirs(SAVE_DIR)
    
    # 全データをロード
    print("Initializing Master Dataset...")
    if not os.path.exists(TRAIN_META): return
    full_df = pd.read_csv(TRAIN_META)
    
    full_ds = ECGDatasetRam(full_df, STAGE2_DIR, CSV_DIR)
    
    # ▼▼▼ 変更箇所ここから ▼▼▼
    
    # ★変更前: ランダムシャッフル (KFold)
    # kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    
    # ★変更後: ID考慮の分割 (GroupKFold)
    gkf = GroupKFold(n_splits=N_FOLDS)
    
    # ★グループ（患者ID）のリストを取得
    groups = full_ds.sample_ids 
    
    # ▲▲▲ 変更箇所ここまで ▲▲▲

    fold_scores = []

    print(f"\n{'='*40}")
    print(f" Starting {N_FOLDS}-Fold CV (GroupKFold)")
    print(f"{'='*40}")

    # ★変更: splitに groups=groups を渡す
    for fold, (train_idx, val_idx) in enumerate(gkf.split(range(len(full_ds)), groups=groups)):
        print(f"\n>>> Fold {fold+1} / {N_FOLDS}")
        
        train_ds = Subset(full_ds, train_idx)
        val_ds = Subset(full_ds, val_idx)
        
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
        
        model = ResNet1d_UNet_Large().to(device)
        
        # Optimizer
        optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        
        # Scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)
        
        use_amp = torch.cuda.is_available()
        scaler = torch.amp.GradScaler('cuda') if use_amp else None
        criterion_raw = nn.MSELoss(reduction='none')

        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0
            
            for inputs, targets, masks, _ in train_loader:
                inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
                optimizer.zero_grad()
                
                if use_amp:
                    with torch.amp.autocast('cuda'):
                        outputs = model(inputs)
                        loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(inputs)
                    loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
                train_loss += loss.item()
            
            avg_train_loss = train_loss / len(train_loader)
            
            # Valid
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, targets, masks, _ in val_loader:
                    inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
                    if use_amp:
                        with torch.amp.autocast('cuda'):
                            outputs = model(inputs)
                            loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                    else:
                        outputs = model(inputs)
                        loss = (criterion_raw(outputs, targets) * masks).sum() / (masks.sum() + 1e-8)
                    val_loss += loss.item()
            avg_val_loss = val_loss / len(val_loader)
            
            scheduler.step(avg_val_loss)

            if epoch % 5 == 0 or avg_val_loss < best_loss:
                 current_lr = optimizer.param_groups[0]['lr']
                 print(f"  [Fold {fold+1} Epoch {epoch+1}] Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f} | LR: {current_lr:.1e}")

            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                patience_counter = 0
                save_path = os.path.join(SAVE_DIR, f"best_model_fold{fold}.pth")
                torch.save(model.state_dict(), save_path)
            else:
                patience_counter += 1
                if patience_counter >= PATIENCE:
                    print(f"  Early stopping at epoch {epoch+1}. Best Val Loss: {best_loss:.6f}")
                    break
        
        print(f"Fold {fold+1} Finished. Best Loss: {best_loss:.6f}")
        fold_scores.append(best_loss)

    print("\n" + "="*40)
    print(" CV FINISHED ")
    print("="*40)
    for i, score in enumerate(fold_scores):
        print(f"Fold {i+1}: {score:.6f}")
    print(f"Average: {np.mean(fold_scores):.6f}")

if __name__ == "__main__":
    run_kfold_training()

Using Device: cuda
Initializing Master Dataset...
Scanning files in C:\Users\fujiw\OneDrive\デスクトップ\ECG_ResNet\stage2...
Found 8793 valid files. Loading ALL into RAM...


Loading Data:  49%|█████▊      | 4293/8793 [01:03<01:38, 45.72it/s]